<a href="https://colab.research.google.com/github/UmbrellaMalware/PythonAnalysisCourse/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание 1 (pandas)

## Задание 1.0

Данные, которые нужно использовать для выполнения задания, возьмите [здесь](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data).

Данные расположены в двух таблицах: **penguins_iter** и **penguiuns_size**. Для этой лабораторной работы вам понадобится только таблица **penguiuns_size**.

**penguins_size** содержит следующую информацию:
* `species`: вид, к которому относится пингвин
* `culmen_length_mm`: длина клюва
* `culmen_depth_mm`: глубина клюва
* `flipper_length_mm`: длина ласт
* `body_mass_g`: масса тела
* `island`: название острова, на котором пингвин обитает
* `sex`: пол пингвина



1. Загрузите файл с данными к себе на Гугл диск и подключите его в Google Colab через `mount.drive`.
2. Выведите общую статистику по всем числовым и нечисловым столбцам с помощью функции `describe()`.
3. Узнайте размеры датасета с помощью функции `shape`.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

In [10]:
%%capture
!wget https://www.dropbox.com/s/mnwjm59duxudj6g/penguins_size.csv

In [11]:
training_data = pd.read_csv('penguins_size.csv')
training_data.describe()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


## Задание 1.1

1. Сколько островов, на которых обитают пингвины, представлено в таблице?
2. Выведите долю пингвинов, обитающих на каждом из островов.

In [26]:
islands = training_data.island.unique()

In [29]:
for i in islands:
  print(f'{i} {training_data.island.value_counts().loc[i]/len(training_data.island)}')


Torgersen 0.1511627906976744
Biscoe 0.4883720930232558
Dream 0.36046511627906974


## Задание 1.2

1. Найдите id пингвина с самым длинным клювом и с самым коротким. Выведите всю информацию о каждом из этих пингвинов в виде таблицы из двух строк.
2. Посчитайте, насколько самый длинный клюв длиннее самого короткого.

In [38]:
training_data[(training_data.culmen_length_mm==training_data.culmen_length_mm.max())|(training_data.culmen_length_mm==training_data.culmen_length_mm.min())]

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
142,Adelie,Dream,32.1,15.5,188.0,3050.0,FEMALE
253,Gentoo,Biscoe,59.6,17.0,230.0,6050.0,MALE


In [39]:
training_data.culmen_length_mm.max() - training_data.culmen_length_mm.min()

27.5

## Задание 1.3

1. Найдите среднее соотношение между длиной и глубиной клюва.
2. Найдите среднее соотношение между массой тела пингвина и длиной его ласт.
3. Постройте корреляционную матрицу для значений столбцов `culmen_length_mm`, `culmen_depth_mm`, `flipper_length_mm`, `body_mass_g`.

In [43]:
(training_data.culmen_length_mm/training_data.culmen_depth_mm).mean()

2.6056485089565236

## Задание 1.4

1. О пингвинах какого вида в таблице больше всего недостающих данных? (Строк с `NaN`)
2. Какой информации преимущественно не хватает?

In [ ]:
# PUT YOUR CODE HERE

## Задание 1.5

По данным из таблицы узнайте, как по размеру клюва, ласт и общей массе тела мужские особи отличаются от женских. Ответ усредните.

In [ ]:
# PUT YOUR CODE HERE

## Задание 1.6

По данным из таблицы узнайте, равномерно ли демографиески (процент мужских и женских особей) пингвины распределены по каждому из островов.

In [ ]:
# PUT YOUR CODE HERE

## Задание 1.7

1. Узнайте, к какому виду принадлежат самые крупные по массе тела пингвины.
2. В каком количестве популяция этого вида представлена на каждом из островов?
3. Равное ли соотношение мужских и женских особей этого вида?

In [ ]:
# PUT YOUR CODE HERE

# Задание 2 (pytorch + matplotlib)

Пусть $f(x) = 3\sin(x) * x$+5. Постройте график $f(x)$ и $\frac{df(x)}{dx}$, для расчета производных воспользуйтесь методом `backward()` и matplotlib для построения графиков. Графики функции и производной разместить на одном изображении.

In [ ]:
# PUT YOUR CODE HERE

# Задание 3 (scipy + matplotlib)

Дано две функции:

\begin{matrix} F(x) = a_{11}x^2 + a_{12}x + a_{13}&(1) \\ P(x) = a_{21}x^2+a_{22}x + a_{23} &(2) \end{matrix}
​	
* На вход программа должна принимать 2 строки (т.е. должен быть реализован ввод с клавиатуры). Каждая строка содержит 3 действительных числа через пробел: коэффициенты $a$.
* Необходимо найти точки экстремума функций (например, через функцию `minimize_scalar()` или `fmin()`) и определить, есть ли у функций общие решения при введенных коэффициентах на некотором диапазоне значений $x$, предусмотрев ситуации, когда решение есть, решений нет и решений бесконечно много. Ответ вывести. В случае, если решения есть (или их бесконечно много), необходимо построить график, на котором построены графики обеих функций, отмечены эти решения и экстремумы функций (другим цветом).


In [ ]:
# PUT YOUR CODE HERE